In [1]:
# Path to dataset
obama_folder = 'C:/Users/aaady/Desktop/Mira/datasets/us_presidential_speeches/democrat/Barack Obama' # FILL OUT FIRST!!!
trump_folder = 'C:/Users/aaady/Desktop/Mira/datasets/us_presidential_speeches/republican/Donald Trump'

In [2]:
import os
import io
import pandas as pd

In [3]:
# Function to run through the dataset folder and add all text to one variable
def get_all_text_files(folder):
    
    files = os.listdir(folder)
    text = ""
    
    for path in files:
        file_path = os.path.join(folder, path)
        file_read = io.open(file_path, mode="r", encoding="utf-8")
        file_text = file_read.read()
        text += file_text
    
    return text

In [4]:
obama_all_text = get_all_text_files(obama_folder)
trump_all_text = get_all_text_files(trump_folder)

In [5]:
speech_dict = {
    'obama': obama_all_text,
    'trump': trump_all_text
}

data_df = pd.DataFrame.from_dict(speech_dict, orient='index')
data_df.columns = ['transcript']
data_df.head()

,transcript
obama,"Thank you, everybody. Thank you. (Applause.) T..."
trump,"Thank you very much, everyone. My administrati..."


In [6]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\([^()]*\)', '', text) # Removes anything between square and regular brackets
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [7]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

,transcript
obama,thank you everybody thank you thank you so mu...
trump,thank you very much everyone my administration...


In [8]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [9]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_clean

,transcript
obama,thank you everybody thank you thank you so mu...
trump,thank you very much everyone my administration...


In [10]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

,abandon,abandoned,abandonment,abandons,abbas,abbott,abbottabad,abdallahs,abe,abebradley,...,zigs,zika,zimbabweans,zimmerman,zinke,zionist,zip,zone,zones,zoning
obama,6,1,1,0,3,0,1,1,0,0,...,1,1,1,2,0,2,1,0,1,0
trump,4,2,1,1,0,1,0,0,6,1,...,0,0,0,0,2,0,0,4,7,1


In [11]:
data = data_dtm.transpose()

In [12]:
data.head()

,obama,trump
abandon,6,4
abandoned,1,2
abandonment,1,1
abandons,0,1
abbas,3,0


In [18]:
# Find the top 30 words said by each president
top_dict = {}
for c in data.columns:
    top = data[c].sort_values(ascending=False).head(30)
    top_dict[c]= list(zip(top.index, top.values))

top_dict['obama']

[('people', 854),
 ('thats', 648),
 ('just', 530),
 ('america', 517),
 ('new', 497),
 ('make', 477),
 ('american', 450),
 ('know', 442),
 ('time', 430),
 ('years', 418),
 ('work', 412),
 ('world', 386),
 ('country', 376),
 ('right', 361),
 ('americans', 352),
 ('want', 351),
 ('like', 349),
 ('weve', 343),
 ('jobs', 341),
 ('going', 316),
 ('economy', 309),
 ('need', 296),
 ('im', 293),
 ('dont', 279),
 ('states', 277),
 ('think', 275),
 ('united', 253),
 ('future', 251),
 ('care', 248),
 ('year', 247)]

In [19]:
top_dict['trump']

[('people', 804),
 ('going', 549),
 ('want', 525),
 ('great', 491),
 ('know', 435),
 ('american', 432),
 ('country', 395),
 ('thank', 392),
 ('just', 345),
 ('dont', 335),
 ('like', 313),
 ('said', 313),
 ('united', 308),
 ('states', 308),
 ('new', 300),
 ('america', 291),
 ('theyre', 285),
 ('thats', 270),
 ('right', 263),
 ('world', 261),
 ('years', 260),
 ('time', 247),
 ('good', 236),
 ('lot', 230),
 ('say', 223),
 ('think', 222),
 ('way', 214),
 ('nations', 200),
 ('really', 185),
 ('nation', 181)]